In [3]:
import pandas as pd
import numpy as np
#PLOT & MATH LIBS
import seaborn as sns
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None  # default='warn'

### Introduction

The primary objective of this notebook is to illustrate several simple techniques which improve the predictive performance of Parkinson's Disease progression (PD) on a population of PD patients. The progression of PD is measured using the PPMI database (https://ida.loni.usc.edu/login.jsp) which consists of UPDRS-III composite scores from 3,812 participants over 13 years. More specifically, we focus on the 3rd subtest of UPDRS-III, which is composed of in clinic motor examinations on patients and controls. This is widely considered (and some quick tests support this claim) to be the more statistically stable of the 4 sub-tests given its mechanical in-person nature. Our results show that a few simple techniques can significantly reduce model variance relative to the predicted outcome, without model selection, tuning, or parameterization. Additionally, we also show another feasible approach alongside ideas for follow up research. 

***Mention: ***
1. The need for improved prediction processes
2. add in the State model 

In [7]:
# Load and Pre-process data set
UPDRS3 = "data/MDS-UPDRS_Part_III_10Jun2024.csv"
patient_status = "data/Participant_Status_03Jun2024.csv"

df3 = pd.read_csv(UPDRS3)
df_pat_stat = pd.read_csv(patient_status) #patient status data
df3 = df3.dropna(subset=['NP3TOT']).reset_index() # will keep for now, might need to include nans
df3['INFODT'] = pd.to_datetime(df3['INFODT'], format="%m/%Y") #reformat INFODT (Assesment Date) to date-time objects
df3['PDSTATE'] =  df3['PDSTATE'].fillna("None")
df3 = df3[["PATNO", "EVENT_ID", "INFODT", "PDSTATE", "PAG_NAME", "NP3TOT"]]

desired_cols_df_pat = {'PATNO', 'COHORT', 'ENROLL_STATUS'}
pat_filtered = df_pat_stat.drop(columns=set(df_pat_stat.columns) - desired_cols_df_pat)
df3_full = pd.merge(df3, pat_filtered, on="PATNO")
df3_full = df3_full[df3_full['ENROLL_STATUS'].isin(['Enrolled', 'Withdrew', 'Complete'])]
df3_full.drop(columns=['ENROLL_STATUS'], inplace=True)
df3_full = df3_full.sort_values(['PATNO', 'INFODT'])

# Partition our data sets
upd3_control = df3_full[df3_full['COHORT'] == 2]
upd3_PD = df3_full[df3_full['COHORT'] == 1]
upd3_PD_nan = upd3_PD[(upd3_PD['PDSTATE'] != 'ON') & (upd3_PD['PDSTATE'] != 'OFF') & (upd3_PD['PAG_NAME'] != 'NUPDR3OF') & (upd3_PD['PAG_NAME'] != 'NUPDR3ON')].reset_index(drop=True)
upd3_PD_off = upd3_PD[(upd3_PD['PDSTATE'] == 'OFF') | (upd3_PD['PAG_NAME'] == 'NUPDR3OF')].reset_index(drop=True)
upd3_PD_on = upd3_PD[(upd3_PD['PDSTATE'] == 'ON') | (upd3_PD['PAG_NAME'] == 'NUPDR3ON')].reset_index(drop=True)

### Methods

##### Model Selection and Logic
As a preamble, this research turned out to be less focused on model selection and parameterization than initially expected. The models themselves are only as good as the data they train on (mostly), and here the discovery is that a clear framing of the PD progression problem as a time-series forcasting problem does a surprising amount of work. On that note, the models used here are placeholders. I selected Random-Forest-Regression (RFR) primarily because my supervisor (who's vastly more knowledgeable and skilled than I) recommended it. I decided to use Ridge-Regression as the inclusion of a linear model to compare to the more complex RFR could add some clues for future research on optimal model selection (though not a true comparison given the lack of a structure hyposthesis test). Additionally, Ridge-Regression could be more robust to the amount of noise inherent in the UPDRS-III dataset due to the slope penalty vs. simple Linear-Regression. 

**Placeholder Models**

- Random Forest Regression, default 5 nodes
- Ridge Regression

##### Data Preparation Process

The objective is to create rows in a wide-dataframe with the following format: `t0 | u0 | t1 | u1 | ... | t_i | u_i`, where `t_i` is the date where score `u_i` was measured. Each row represents the `[date | score]` entries for a given patient. The following algorithm performs this task:

1. Given the non-uniform number of respective patient entries (some have 1, others more than 20) and to retain as much data as possible for our models we decided to bound the number of observations per patient to 3 (`i = 3`), which allowed us to keep patients with at least 3 and those with many more entries. So each patient's entries will be: `t0 | u0 | t1 | u1 | t_2 | u_2`

   1.  This is done by using the individual patients entries to create a calendar, which is then segmented into 3 non-overlapping intervals, each greater than 6 months. The 6 month limit was chosen as it allows the disease to progress and therefore reduces overall measurement noise. The interval time delta is calculated as (end - start) / 3, where both end and start are drawn from the patients first and last visit INFODT (or date of examination). The patients individual calendar is then created by iteratively adding this time delta to the start date. This creates 4 dates, and thereby 3 intervals.

2. From here, the patients 3 entries, `u0 | u1| u2` are produced as follows (through the functions `process_mean_intervals` and `process_rand_intervals`): 
   1. For the random basline, each of the 3 entries are randomly drawn from it's respective interval [Note 1]
   2. The improved model takes the mean UPDRS-III score over the respective intervals as its entries

3. For both methods, the date attached to these intervals,`t_i`, is the median date of each interval, which is calculated by adding 1/2 the time delta to each of the intervals start dates.
4. Finally, the returned dataframes are converted to a wide format such that each row represents the 3 selected (or mean aggregated) INFODT-score combinations for a given PATNO.


**Note 1**: It's also possible to randomly select 3 sequential patient entries such that they are taken at least 6 months from each other. But this wouldn't allow us to best measure the improvment in model performance using the mean over intervals.

**Note 2**: (Rough) Function documentation for helper functions
- `median_date_calc` : returns the median date of a grouped object
- `interpolate_same_month` : Returns the specified computation (or random selection) on a grouped object. The grouping is done to filter same PATNO's and equivalent dates for the same PATNO, which for this analysis adds noise
- `pivot_wide` : Returns the passed dataframe in wide format, some pre-processing is done in the `process_mean_intervals` and `process_rand_intervals` functions

In [9]:
# Data Processing and Helper Functions
def median_date_calc(date_group: pd.DataFrame, interval: pd.Timedelta) -> list:
    dates = date_group['INFODT'].tolist()
    date1 = dates[0] + interval / 2
    date2 = dates[1] + interval / 2
    date3 = dates[2] + interval / 2
    return [(date1-date1).days, (date2-date1).days, (date3-date1).days]

def interpolate_same_month(df: pd.DataFrame, method = 'rand') -> pd.DataFrame:
    # Select maximum score from same month measurements
    temp_df = df.copy()
    temp_df['YEAR_MONTH'] = temp_df['INFODT'].dt.to_period('M')

    #takes random selctions/maximum/minimum/mean of values which share same month and year
    if method == 'rand':
        sample = temp_df.groupby(['PATNO', 'YEAR_MONTH'])['NP3TOT'].apply(lambda x: x.sample(1)).reset_index()
        result = pd.merge(temp_df, sample, on=['PATNO', 'YEAR_MONTH', 'NP3TOT'])

        result.drop(columns=['YEAR_MONTH', 'level_2'], inplace=True)
        return result.drop_duplicates(subset=['PATNO', 'INFODT']).reset_index(drop=True)

    elif method == 'min':
        #result = pd.merge(temp_df, temp_df.groupby(['PATNO', 'YEAR_MONTH'])['NP3TOT'].min(), on=['PATNO', 'YEAR_MONTH', 'NP3TOT'])
        temp_df['NP3TOT'] = temp_df.groupby(['PATNO', 'YEAR_MONTH'])['NP3TOT'].transform('min')
    elif method == 'max':
        #result = pd.merge(temp_df, temp_df.groupby(['PATNO', 'YEAR_MONTH'])['NP3TOT'].max(), on=['PATNO', 'YEAR_MONTH', 'NP3TOT'])
        temp_df['NP3TOT'] = temp_df.groupby(['PATNO', 'YEAR_MONTH'])['NP3TOT'].transform('max')
    else:
        #result = pd.merge(temp_df, temp_df.groupby(['PATNO', 'YEAR_MONTH'])['NP3TOT'].mean(), on=['PATNO', 'YEAR_MONTH', 'NP3TOT'])
        temp_df['NP3TOT'] = temp_df.groupby(['PATNO', 'YEAR_MONTH'])['NP3TOT'].transform('mean')

    
    temp_df.drop(columns=['YEAR_MONTH'], inplace=True)
    temp_df.reset_index(drop=True, inplace=True)
    return temp_df.drop_duplicates(subset=['PATNO', 'INFODT'])

def pivot_wide(df: pd.DataFrame, cols: int = 3) -> pd.DataFrame:

    df['time_index'] = (
            df.groupby('PATNO')['INFODT']
            .rank(method='first')
            .astype(int)-1
        ) 
    
    time_df = df[['PATNO', 'INFODT', 'time_index']]

    score_wide = df.pivot(
        index='PATNO',
        columns='time_index',
        values='NP3TOT'
    ).reset_index()

    time_wide = time_df.pivot(
        index='PATNO',
        columns="time_index",
        values='INFODT'
    ).reset_index()

    score_wide.columns = ['PATNO'] + [f'u{i}' for i in range(cols)] # resets score column names correctly
    time_wide.columns = ['PATNO'] + [f't{i}' for i in range(cols)] # resets time_index column names accordingly

    merged = pd.merge(score_wide, time_wide, on='PATNO')
    merged.drop(columns='PATNO', inplace=True)

    new_cols = []
    for i in range(cols): # re order columns for ML piplines
        new_cols.append(f't{i}')
        new_cols.append(f'u{i}')

    merged = merged[new_cols]

    return merged.drop(columns='t0')


# keep_small_intervals flag allows us to keep some entries with <6mo intervals
# assumes process_same_dates has already been calles
def process_mean_intervals(df: pd.DataFrame, blocks: int = 3, keep_small_intervals = True) -> pd.DataFrame:
    patnos = df['PATNO'].unique().tolist()
    result = pd.DataFrame(columns=['PATNO', 'INFODT', 'NP3TOT'])
    for patno in patnos:
        df_pat = df[df['PATNO'] == patno]
        start = df_pat['INFODT'].min()
        end = df_pat['INFODT'].max()
        diff = end - start

        if (diff < pd.Timedelta(days=180)) or (df_pat.index.size < 3):
            continue

        interval = diff // blocks

        # drop small intervals < 6mo
        if not keep_small_intervals and interval < pd.Timedelta(days=180):
            continue

        group_counts = df_pat.groupby(pd.Grouper(key='INFODT', freq= f'{interval.days+1}D'))['NP3TOT'].count()
        temp = df_pat.groupby(pd.Grouper(key='INFODT', freq= f'{interval.days+1}D'))['NP3TOT'].mean().reset_index()
        temp['group_counts'] = group_counts.values
        # checks for Failed Time Group Function 
        if temp['NP3TOT'].isnull().values.any():
            continue
        
        dates = median_date_calc(df_pat.groupby(pd.Grouper(key='INFODT', freq= f'{interval.days}D'))['NP3TOT'].mean().reset_index(), interval)

        # this logic will create some entries with < 6mo intervals, but this is only due to the abover grouper function using a slightly different calendar
        temp['INFODT'] = dates
        temp['PATNO'] = patno

        result = pd.concat([result, temp], ignore_index=True)

    return result

def process_random_intervals(df: pd.DataFrame, blocks: int = 3, keep_small_intervals = True) -> pd.DataFrame:
    patnos = df['PATNO'].unique().tolist()
    patnos = df['PATNO'].unique().tolist()
    result = pd.DataFrame(columns=['PATNO', 'INFODT', 'NP3TOT'])
    for patno in patnos:
        df_pat = df[df['PATNO'] == patno]
        start = df_pat['INFODT'].min()
        end = df_pat['INFODT'].max()
        diff = end - start

        if (diff < pd.Timedelta(days=180)) or (df_pat.index.size < 3):
            continue

        interval = diff // blocks

        # drop small intervals < 6mo
        if not keep_small_intervals and interval < pd.Timedelta(days=180):
            continue

        temp = (
            df_pat.groupby(pd.Grouper(key='INFODT', freq=f'{interval.days+1}D'))['NP3TOT']
            .apply(lambda x: x.sample(n=1, random_state=1) if len(x) > 0 else None)
            .dropna()  # Drop groups where no sample was taken (i.e., empty groups)
            .reset_index()
        )

        # checks for Failed Time Group Function // when the groupby function can't return 3 groups (like it should)
        if temp.index.size != 3:
            #print("Failed Time Group Function")
            continue

        dates = median_date_calc(df_pat.groupby(pd.Grouper(key='INFODT', freq= f'{interval.days}D'))['NP3TOT'].mean().reset_index(), interval)

        # this logic will create some entries with < 6mo intervals, but this is only due to the abover grouper function using a slightly different calendar
        temp['INFODT'] = dates
        temp['PATNO'] = patno
        temp.drop(columns=['level_1'], inplace=True)

        result = pd.concat([result, temp], ignore_index=True)

    return result

### Results

<img src="results.png" alt="isolated" width="800" class = "center"/>
![alt text](results.png "Tabulated Results, all values in RMSE")

In [10]:
# Model testing function and pipeline

# Imports
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline

# Pipline Data Sets
nan = interpolate_same_month(upd3_PD_nan, 'rand')
off = interpolate_same_month(upd3_PD_off, 'rand')
on = interpolate_same_month(upd3_PD_on, 'rand')
control = interpolate_same_month(upd3_control, 'rand')

#BASELINES RANDOM
base_nan_rand = np.std(nan['NP3TOT'])
base_off_rand = np.std(off['NP3TOT'])
base_on_rand  = np.std(on['NP3TOT'])
base_control_rand = np.std(control['NP3TOT'])

PD_nan_rand = pivot_wide(process_random_intervals(nan, 3, keep_small_intervals=True))
PD_off_rand = pivot_wide(process_random_intervals(off, 3, keep_small_intervals=True))
PD_on_rand = pivot_wide(process_random_intervals(on, 3, keep_small_intervals=True))
control_rand = pivot_wide(process_random_intervals(control, 3, keep_small_intervals=True))

#DE-NOISED
nan_mean = interpolate_same_month(upd3_PD_nan, 'mean')
off_mean = interpolate_same_month(upd3_PD_off, 'mean')
on_mean = interpolate_same_month(upd3_PD_on, 'mean')
control_mean = interpolate_same_month(upd3_control, 'mean')

base_nan_mean = np.std(nan_mean['NP3TOT'])
base_off_mean = np.std(off_mean['NP3TOT'])
base_on_mean  = np.std(on_mean['NP3TOT'])
base_control_mean = np.std(control_mean['NP3TOT'])

PD_nan_mean = pivot_wide(process_mean_intervals(nan_mean, 3, keep_small_intervals=True))
PD_off_mean = pivot_wide(process_mean_intervals(off_mean, 3, keep_small_intervals=True))
PD_on_mean = pivot_wide(process_mean_intervals(on_mean, 3, keep_small_intervals=True))
control_mean = pivot_wide(process_mean_intervals(control_mean, 3, keep_small_intervals=True))

/var/folders/3f/j_4mwg8n6c137n0zbqfb_rbc0000gn/T/ipykernel_7171/1375517764.py:151: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, temp], ignore_index=True)
/var/folders/3f/j_4mwg8n6c137n0zbqfb_rbc0000gn/T/ipykernel_7171/1375517764.py:151: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, temp], ignore_index=True)
/var/folders/3f/j_4mwg8n6c137n0zbqfb_rbc0000gn/T/ipykernel_7171/1375517764.py:151: FutureWarning: The behavior of DataFrame concatenat

In [11]:
# Model Caller Functions
def test_models(df:pd.DataFrame, base_line: float, identifier: str = "DEFAULT", folds = 5):
    X = df.iloc[: , : len(df.columns)-1]
    y = df.iloc[: , len(df.columns)-1 : ]

    pipe_rf = Pipeline(steps=[('model', RandomForestRegressor())])
    pipe_ridge = Pipeline(steps=[('model', Ridge())])
    # .values will give the values in a numpy array (shape: (n,1))
    # .ravel will convert that array shape to (n, ) (i.e. flatten it)
    score_test_rf = -1 * cross_val_score(pipe_rf, X, y.values.ravel(), cv=folds, scoring="neg_root_mean_squared_error")
    score_test_ridge = -1 * cross_val_score(pipe_ridge, X, y.values.ravel(), cv=folds, scoring="neg_root_mean_squared_error")

    print(f"Testing {identifier} Model")
    print("----------------------------------------------------------------")
    print(f"STD(NP3TOT) {identifier}: {base_line}")
    print("----------------------------------------------------------------")

    print(f"Startings models, Simple Cross Validation, k = {folds}, VS std(UPDRS): \n")
    print(" Ridge Regression RMSE by fold: ", score_test_ridge)
    print(" Random Forest RMSE by fold: ", score_test_rf, "\n")
    print(f" Ridge Regression mean RMSE: {score_test_ridge.mean()}", '\n', f"Random Forest, default 5-nodes, mean RMSE: {score_test_rf.mean()}")

    return [score_test_rf.mean(), score_test_ridge.mean()]

def compare_models(df_mean: pd.DataFrame, df_rand: pd.DataFrame, base_mean: float, base_rand: float):
    mean = test_models(df_mean, base_mean, "MEAN")
    rand = test_models(df_rand, base_rand, "Random Selection")

    print("----------------------------------------------------------------")
    print("Comparing Models")
    print("----------------------------------------------------------------")
    print(f"Mean Model: {mean}")
    print(f"Random Model: {rand}")

### Conclusion

##### Thoughts on follow up research
- Model Selection: Here we used only placeholder models with no parameterization. The next phase would be to research and test other models. 
- New Features: The only feature in our analysis is time. Medical imaging, Age, and the other 3 UPDRS-III scores are obvious candidates to include here